In [2]:
import sys
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.signal import detrend
from scipy.stats import gaussian_kde, weightedtau, kendalltau, pearsonr, spearmanr, rankdata
%matplotlib inline

In [1]:
HOMEDIR = Path('/scistor/ivm/jsn295/')
sys.path.append(str(HOMEDIR))
from Weave.src.utils import agg_time, chi
from SubSeas.helper_functions import monthtoseasonlookup

NameError: name 'Path' is not defined

In [ ]:
response = xr.open_dataarray(HOMEDIR / 'processed/t2m_europe.anom.nc')
clusterfield = xr.open_dataarray(HOMEDIR / 'clusters/t2m-q095.nc').sel(nclusters = 14)
reduced = response.groupby(clusterfield).mean('stacked_latitude_longitude')
reduced = reduced.sel(clustid = 9)

In [ ]:
reduced.plot()

### Look at the sst patterns and z300 patterns

In [ ]:
sstcorr = xr.open_dataarray(HOMEDIR / 'correlation_roll_spearman/sst_nhplus.5.corr.nc', decode_times = False)
z300corr = xr.open_dataarray(HOMEDIR / 'correlation_roll_spearman/z300_nhmin.5.corr.nc', decode_times = False)

In [ ]:
sstcorr[0].plot()

In [ ]:
sstcorr.sel(latitude = 45, longitude = -30, method = 'nearest').plot()

In [ ]:
sstcorr.sel(latitude = 37, longitude = -50, method = 'nearest').plot()

In [ ]:
sstcorr.sel(latitude = 43, longitude = 4.3, method = 'nearest').plot()

In [5]:
sstcoords = pd.DataFrame({'latitude':[37,45,43],
                      'longitude':[-50,-30,4.3]},
                     index = pd.Index(['athot','atcold','medhot'], name = 'location'))

In [ ]:
sstcoords

In [ ]:
z300corr[4].plot()

In [ ]:
z300corr[0].plot()

In [ ]:
z300corr.sel(latitude = 50, longitude = 60, method = 'nearest').plot()

In [ ]:
z300corr.sel(latitude = 30, longitude = 60, method = 'nearest').plot()

In [6]:
z300coords = pd.DataFrame({'latitude':[50,30],
                      'longitude':[60,60]},
                     index = pd.Index(['kzlow','irhigh'], name = 'location'))

In [ ]:
z300coords

In [7]:
coords = pd.concat([sstcoords, z300coords], axis = 0, keys = ['sst','z300'], names = ['variable','location'])

In [ ]:
#sstcorr.close()
#z300corr.close()
coords

### Take these coords to extract unaggregated anomalie timeseries
And then investigate their (lagged / aggregated) properties and dependencies with the reduced response

In [8]:
data = {}
for ind, cords in coords.groupby(['variable','location']):
    array =  xr.open_dataarray((HOMEDIR / 'processed/sst_nhplus.anom.nc') if ind[0] == 'sst' else (HOMEDIR / 'processed/z300_nhmin.anom.nc'))
    #print(array.sel(latitude = cords['latitude'][0], longitude = cords['longitude'][0], method = 'nearest').values)
    data.update({ind:array.sel(latitude = cords['latitude'][0], longitude = cords['longitude'][0], method = 'nearest').values})
    data.update({('time',''):array.coords['time'].values})
    array.close()

In [9]:
test = pd.DataFrame(data)
test = test.set_index('time')
test.loc[:,('t2m','westeur')] = reduced.values

First check, concurrent and summer only

In [ ]:
summer = test.loc[monthtoseasonlookup(test.index.month) == 'JJA',:]

In [ ]:
summer.hist()

So clearly the geopotential heigths are not really normally distributed within summer. Potentially due to a large trend?

In [ ]:
summer.loc[:,('z300','irhigh')].plot()

In [ ]:
summer_det = summer.copy()
summer_det.loc[:,:] = detrend(summer_det, axis = 0)
summer_det.hist()

In [ ]:
np.round(np.corrcoef(summer, rowvar = False), 3)

In [ ]:
summer_det.plot.scatter(x = -4, y = -1)

### Now we go back to the full timeseries to make a lagging + aggregating + dependence

In [10]:
temp = test.copy()
tempxr = xr.DataArray(temp)

In [11]:
aggregated = agg_time(tempxr, ndayagg = 5, rolling = True)
agg = aggregated[aggregated.time.dt.season == 'JJA',:].to_pandas()

In [ ]:
agg.plot.scatter(x = -2, y = -1)

In [ ]:
np.round(np.corrcoef(agg,rowvar = False), 4)

In [ ]:
bi_var = agg.loc[:,(slice(None),['kzlow','westeur'])]
bi_var_transp = bi_var.values.T
z = gaussian_kde(dataset=bi_var_transp)(bi_var_transp)
idx = z.argsort()

In [ ]:
x = bi_var_transp[0,:][idx]
y = bi_var_transp[1,:][idx]
z = z[idx]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x, y, c=z, s=10, edgecolor=None)
plt.show()

In [ ]:
def density_plot(xy, ax, xlabel = None, ylabel = None, title = None, annotations: dict = None):
    """
    shape (2,n_obs), with x on [0,:] and y on [1,:]
    """
    z = gaussian_kde(dataset=xy)(xy)
    idx = z.argsort()
    x = xy[0,:][idx]
    y = xy[1,:][idx]
    z = z[idx]
    ax.scatter(x, y, c=z, s=10, edgecolor=None)
    if not xlabel is None:
        ax.set_xlabel(xlabel)
    if not ylabel is None:
        ax.set_ylabel(ylabel)
    if not title is None:
        ax.set_title(title)
    if not annotations is None:
        ys = list(np.arange(0.1, 0.1* len(annotations), 0.1))
        for key, val in annotations.items():
            ax.annotate(s = f'{key},{val}', xy = (0.01,ys.pop()), xycoords = 'axes fraction', color = 'r', fontweight = 'bold')
    return ax

# Ranking by precursor (standard postitive), ranking by precursor (choice), mixture, unweighted, pearson, spearman
def rankdirection(x,y):
    """
    Returns a rank array (0 most important) with emphasis on negative x when negative relation
    emphasis on positive x when positive overall relation
    """
    ranks = rankdata(x, method = 'ordinal')
    if pearsonr(x = x, y = y)[0] < 0:
        return ranks
    else:
        return ranks.max() - ranks

def generate_corrs(x,y):
    corrs = {}
    corrs['pearson'] = pearsonr(x = x, y = y)[0]
    corrs['spearman'] = spearmanr(a = x, b = y)[0]
    corrs['tau'] = kendalltau(x,y)[0]
    corrs['tauw_avg'] = weightedtau(x = x, y = y, rank=True)[0]
    corrs['tauw_xps'] = weightedtau(x = x, y = y, rank=None)[0]
    corrs['tauw_xch'] = weightedtau(x = x, y = y, rank=rankdirection(x = x, y = y))[0]
    return {key: np.round(item, 4) for key, item in corrs.items()}

# Aggregation controls. For a single aggregation I want to make series of scatterplots at multiple lags. For one variable.
def lag_and_plot_pair(aggarr, laglist, x = ('z300','kzlow'), y = ('t2m','westeur'), detr = False):
    """
    Is supplied with the aggregated array (outcome of agg_time), does lagging with xarray
    laglist is in days 
    Then subsetting and plotting the scatter    
    """
    aggarr = aggarr.copy()
    oritimeaxis = aggarr.coords['time']
    start = pd.Timestamp(oritimeaxis[0].values).strftime('%Y-%m-%d')
    yvals = aggarr.sel(dim_1 = y)[aggarr.time.dt.season == 'JJA']
    fig, axes = plt.subplots(ncols = len(laglist), figsize = (4*len(laglist),3))
    for lag in laglist:
        aggarr['time'] = oritimeaxis - pd.Timedelta(str(lag) + 'D')
        print(f'lag: {lag} assigns {pd.Timestamp(aggarr["time"][0].values).strftime("%Y-%m-%d")} to xvalue at {start}')
        xvals = aggarr.sel(dim_1 = x).reindex_like(yvals)
        xy = np.vstack([xvals.values, yvals.values])
        if detr:
            xy = detrend(xy, axis = 1)
        corrs = generate_corrs(x = xy[0,:], y = xy[1,:])
        axes[laglist.index(lag)] = density_plot(xy, ax = axes[laglist.index(lag)], xlabel = x, ylabel = y, title = f'lag {lag} [days], detr: {detr}', annotations = corrs)
        
    plt.show()

In [ ]:
aggregated = agg_time(tempxr, ndayagg = 5, rolling = True)

In [ ]:
lag_and_plot_pair(aggregated, laglist = [0,-1,-3,-5,-7,-10], x=('z300','irhigh'), detr =True)

In [ ]:
lag_and_plot_pair(aggregated, laglist = [0,-1,-3,-5,-7,-10], x=('z300','kzlow'), detr = True)

In [ ]:
lag_and_plot_pair(aggregated, laglist = [-1,-3,-5,-7,-10,-20,-30], x=('sst','atcold'), detr = True)

Interestingly you see the rolling aggregation in the fact that there are trajectories in the outliers. I wonder how the plots will change with different aggregation periods and with a detrending. Perhaps the dense yellow blob disappears. Detrending will have most influence if the trend is different in both (sign or absent/present)

Let's check detrending

In [ ]:
lag_and_plot_pair(aggregated, laglist = [-1,-3,-5,-7,-10,-20,-30], x=('sst','athot'), detr = True)

In [ ]:
lag_and_plot_pair(aggregated, laglist = [-1,-3,-5,-7,-10,-20,-30], x=('sst','medhot'), detr = True)

In [ ]:
column = 0
print(kendalltau(agg.iloc[:,column],agg.iloc[:,-1]))
print(weightedtau(agg.iloc[:,column],agg.iloc[:,-1]))
print(np.corrcoef(agg.iloc[:,[column,-1]], rowvar = False))

In [ ]:
# Ranking by the temperature only, or both?
ranks = rankdata(agg.iloc[:,-1], method = 'ordinal')
ranks = ranks.max() - ranks # Zero is the largest element
ranksx = rankdata(agg.iloc[:,column], method = 'ordinal') # Negative relation for the variable so we want extreme low to be most important.
print(weightedtau(x = agg.iloc[:,column], y = agg.iloc[:,-1], rank=ranks))
print(weightedtau(x = agg.iloc[:,-1], y = agg.iloc[:,column], rank=None)) # From documentation: if rank = None then Elements with larger x values will have more importance. Positive response tail
print(weightedtau(x = agg.iloc[:,column], y = agg.iloc[:,-1], rank=None)) # Letting the precursor be the importance variable (positive tailed)
print(weightedtau(x = agg.iloc[:,column], y = agg.iloc[:,-1], rank=ranksx)) # Explicit precursor importantce (negative tailed)
print(weightedtau(x = agg.iloc[:,-1], y = agg.iloc[:,column], rank=ranksx)) # Explicit precursor importantce (negative tailed)
print(weightedtau(x = agg.iloc[:,-1], y = agg.iloc[:,column], rank=True)) 

In [ ]:
corrs = generate_corrs(x = agg.iloc[:,column], y = agg.iloc[:,-1])
rankdirection(x = agg.iloc[:,column], y = agg.iloc[:,-1])
aggregated = agg_time(tempxr, ndayagg = 5, rolling = True)
lag_and_plot_pair(aggregated, laglist = [0,-1,-3,-5,-7,-10])

In [ ]:
lag_and_plot_pair(aggregated, laglist = [0,-1,-3,-5,-7,-10], x = ('sst','atcold'), detr = True)

In [ ]:
ranksx

In [ ]:
# In the end some sort of matrix plot with aggregations and lags? One variable, multiple measures.
agg.iloc[:,column]

### Chi dependence thresholds

In [36]:
chi(agg.loc[:,('t2m','westeur')], agg.loc[:,('sst','medhot')], full = True, qlim = (0.05,0.95))

(array([0.65947536, 0.646491  , 0.62522453, 0.61224429, 0.59898921,
        0.59175865, 0.57781643, 0.56380256, 0.54705483, 0.54808776,
        0.54987008, 0.55013719, 0.54500959, 0.53921872, 0.53478583,
        0.53712669, 0.52586707, 0.53350413, 0.52980668, 0.53994059,
        0.54250607, 0.55158931, 0.55378536, 0.56350606, 0.566799  ,
        0.56810724, 0.57138624, 0.57156392, 0.57126828, 0.56330402,
        0.56225761, 0.55490884, 0.5601311 , 0.55215734, 0.54620383,
        0.54677057, 0.53791825, 0.53665048, 0.53501394, 0.52525636,
        0.52516453, 0.53016406, 0.5336284 , 0.52590696, 0.52972453,
        0.53207886, 0.52981009, 0.52827262, 0.52746362, 0.53056385,
        0.53226497, 0.52728266, 0.52514497, 0.51946713, 0.51341757,
        0.51238643, 0.51101546, 0.51256502, 0.51051919, 0.50921926,
        0.51310596, 0.51226021, 0.5144395 , 0.51180053, 0.51449707,
        0.50767529, 0.50386704, 0.49605304, 0.49973476, 0.49461356,
        0.48648684, 0.48531732, 0.48502273, 0.48

In [49]:
chi(agg.loc[:,('t2m','westeur')], agg.loc[:,('sst','medhot')], full = True, qlim = (0.05,0.95))

(array([0.65947536, 0.646491  , 0.62522453, 0.61224429, 0.59898921,
        0.59175865, 0.57781643, 0.56380256, 0.54705483, 0.54808776,
        0.54987008, 0.55013719, 0.54500959, 0.53921872, 0.53478583,
        0.53712669, 0.52586707, 0.53350413, 0.52980668, 0.53994059,
        0.54250607, 0.55158931, 0.55378536, 0.56350606, 0.566799  ,
        0.56810724, 0.57138624, 0.57156392, 0.57126828, 0.56330402,
        0.56225761, 0.55490884, 0.5601311 , 0.55215734, 0.54620383,
        0.54677057, 0.53791825, 0.53665048, 0.53501394, 0.52525636,
        0.52516453, 0.53016406, 0.5336284 , 0.52590696, 0.52972453,
        0.53207886, 0.52981009, 0.52827262, 0.52746362, 0.53056385,
        0.53226497, 0.52728266, 0.52514497, 0.51946713, 0.51341757,
        0.51238643, 0.51101546, 0.51256502, 0.51051919, 0.50921926,
        0.51310596, 0.51226021, 0.5144395 , 0.51180053, 0.51449707,
        0.50767529, 0.50386704, 0.49605304, 0.49973476, 0.49461356,
        0.48648684, 0.48531732, 0.48502273, 0.48

In [21]:
agg.columns

MultiIndex([( 'sst',  'atcold'),
            ( 'sst',   'athot'),
            ( 'sst',  'medhot'),
            ('z300',  'irhigh'),
            ('z300',   'kzlow'),
            ( 't2m', 'westeur')],
           names=['dim_1_level_0', 'dim_1_level_1'])